In [5]:
from keras.models import Sequential, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers.crf import CRF
from keras_contrib.utils import save_load_utils
from keras_contrib.metrics import crf_accuracy
from keras_contrib.losses import crf_loss
from keras.utils import to_categorical
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import os

from fastText import load_model


In [2]:
wiki = load_model("./data/wiki.en.bin")
wiki_dic = set(wiki.get_words())

In [3]:
def get_json(fn):
    with open(fn, 'r', encoding='utf-8') as f:
        text = ''.join(f.readlines())
    return json.loads(text)


class Dataprocessor:
    def __init__(self):
        self.id2tag = {0: 'PAD', 1: 'B0', 2: 'B1', 3: 'B2', 4: 'B3', 5: 'B4', 6: 'B-1', 
                       7: 'B-2', 8: 'B-3', 9: 'B-4', 10: 'M', 11: 'E'}
        self.tag2id = {}
        for i in self.id2tag:
            self.tag2id[self.id2tag[i]] = i
        self.max_paragraph = 0
    
    def getid(self, tag):
        return self.tag2id[tag]

    def gettag(self, id):
        return self.id2tag[id]

    def section_split(self, datalist):
        for js in datalist:
            del js["simple"]
            new_full = []
            for i in js["full"]["sectionContents"]:
                if len(i["text"]) == 0 or i["isContentSection"] == False:
                    continue
                i["text"] = i["text"].split('\n')
                new_full.append(i)
            js["full"]["sectionContents"] = new_full

    def tagging(self, data):
        for js in data:
            last = None
            for i in js["full"]["sectionContents"]:
                i["tags"] = []
                i["raw_tags"] = []
                for id, x in enumerate(i["text"]):
                    if last is None:
                        last = i["depth"]
                    th = i["depth"] - last
                    if id == len(i["text"]) - 1:
                        tag = "E"
                    else:
                        tag = "M"
                    if id == 0:
                        tag = "B%d"%(th)
                    i["tags"].append(self.getid(tag))
                    i["raw_tags"].append(tag)
                last = i["depth"]
    
    def load_data(self, jsonfilelist):
        article_texts = []
        article_tags = []
        for f in jsonfilelist:
            print('loading %s...' % (os.path.basename(f)))
            data = get_json(f)
            self.section_split(data)
            self.tagging(data)
            for k, article in enumerate(data):
                print(k)
                paragraphs = []
                tags = []
                for sec in article['full']['sectionContents']:
                    
                    rtn = []
                    for i in sec['text']:
                        rtn.append(self.str2vec(i))
                        
                    tmp = np.mean(np.array(rtn), axis=0)
                    paragraphs.append(tmp)
                    tags.extend(sec['tags'])
                article_texts.append(paragraphs)
                article_tags.append(tags)
                
        return article_texts, article_tags
                
        

    def str2vec(self, test_str):
        
        data = word_tokenize(test_str)
        rtn_vec = []
        for i in data:
            if i not in wiki_dic:
#                 print("The word cannot be found!")
                continue
            rtn_vec.append(wiki.get_word_vector(i))
        if len(rtn_vec) == 0:
            return [0] * 300
        return np.mean(np.array(rtn_vec), axis= 0)

In [6]:
data_path = './data/'
filelist = [(data_path + '%d.json' % i) for i in range(5)]
processor = Dataprocessor()
# train_texts, train_tags, train_rawtags = processor.load_data(filelist)
train_vec, train_tags = processor.load_data(filelist)


loading 0.json...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
loading 1.json...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [7]:
print(np.array(train_vec).shape, np.array(train_tags).shape)
print(np.array(train_vec[0]).shape, np.array(train_tags[0]).shape)
print(type(train_vec[0]), type(train_tags[0]))


(1147,) (1147,)
(5, 300) (14,)
<class 'list'> <class 'list'>


In [8]:
# from bert_utils import get_all_features
# import os
# BERT_BASE = os.path.join(os.getcwd(), 'bert/bert_model/uncased_L-12_H-768_A-12')

# sample_sum = 200

# bert_config_file = os.path.join(BERT_BASE, 'bert_config.json')
# vocab_file = os.path.join(BERT_BASE, 'vocab.txt')
# bert_checkpoint = os.path.join(BERT_BASE, 'bert_model.ckpt')
    
# feature = get_all_features(train_texts[0:sample_sum], bert_config_file, vocab_file, bert_checkpoint)
# print(len(feature))

# 8520: paragraphs
# 6000: 
# 0: item
# 1: 14.. sentence per item
# 2: 768 vec

In [9]:
class LSTMmodel:
    def __init__(self, input_length, para_emb_dim, num_tags, hidden_dim=200, dropout=0.5):
        self.num_tags = num_tags
        self.model = Sequential()
        self.model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True), input_shape=(input_length, para_emb_dim)))
        self.model.add(Dropout(dropout))
        # self.model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True), input_shape=(input_length, para_emb_dim)))
        # self.model.add(Dropout(dropout))
        self.model.add(TimeDistributed(Dense(self.num_tags)))
        crf = CRF(self.num_tags)
        self.model.add(crf)
        self.model.compile('rmsprop', loss=crf_loss, metrics=[crf_accuracy])
    
    def save_model(self, filepath):
        save_load_utils.save_all_weights(self.model, filepath)
    
    def restore_model(self, filepath):
        save_load_utils.load_all_weights(self.model, filepath)
        
    def train(self, trainX, trainY, batch_size=32, epochs=10, validation_split=0.1, verbose=1):
        return self.model.fit(trainX, np.array(trainY), batch_size=batch_size, epochs=epochs, 
                             validation_split=validation_split, verbose=verbose)

In [10]:
INPUT_LENGTH = 1000
PARAGRAPH_EMB_DIM = 300
NUM_TAGS = 12

model = LSTMmodel(INPUT_LENGTH, PARAGRAPH_EMB_DIM, NUM_TAGS)
model.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 1000, 400)         801600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 400)         0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1000, 12)          4812      
_________________________________________________________________
crf_1 (CRF)                  (None, 1000, 12)          324       
Total params: 806,736
Trainable params: 806,736
Non-trainable params: 0
_________________________________________________________________


In [12]:
# load data
import numpy as np



X, rawY = [], [] # X is 3D: article, paragraph, embedding; Y is 2D: article, paragraph
for f, t in zip(train_vec, train_tags):

    while len(f) < INPUT_LENGTH:
        f.append(np.zeros(PARAGRAPH_EMB_DIM))
        t.append(0)
    f = f[0:INPUT_LENGTH]
    t = t[0:INPUT_LENGTH]
    X.append(f)
    rawY.append(t)

Y = [to_categorical(y, num_classes=NUM_TAGS) for y in rawY] # Y is now 3D

data_size = len(X)
train_size = int(data_size * 0.9)
trainX, trainY = X[:train_size], Y[:train_size]
testX, testY = X[train_size:], Y[train_size:]

# print(len(trainX),len(trainY))
# print(len(trainX[0]),len(trainY[0]))

# for i in range(len(trainX)):
#     for j in range(len(trainX[i])):
#         print(len(trainX[i][j]),len(trainY[i][j]))

In [13]:
# train
history = model.model.fit(np.array(trainX), np.array(trainY), batch_size=32, epochs=10, validation_split=0.1)

Train on 928 samples, validate on 104 samples
Epoch 1/10
928/928 [==============================] - 138s 148ms/step - loss: 0.4383 - crf_accuracy: 0.8789 - val_loss: 0.1698 - val_crf_accuracy: 0.9689
Epoch 2/10
928/928 [==============================] - 135s 146ms/step - loss: 0.2392 - crf_accuracy: 0.9209 - val_loss: 0.1341 - val_crf_accuracy: 0.9665
Epoch 3/10
928/928 [==============================] - 135s 145ms/step - loss: 0.1299 - crf_accuracy: 0.9631 - val_loss: 0.0895 - val_crf_accuracy: 0.9714
Epoch 4/10
928/928 [==============================] - 135s 145ms/step - loss: 0.1257 - crf_accuracy: 0.9647 - val_loss: 0.1009 - val_crf_accuracy: 0.9698
Epoch 5/10
928/928 [==============================] - 134s 144ms/step - loss: 0.1296 - crf_accuracy: 0.9640 - val_loss: 0.0795 - val_crf_accuracy: 0.9730
Epoch 6/10
928/928 [==============================] - 136s 146ms/step - loss: 0.1193 - crf_accuracy: 0.9648 - val_loss: 0.1017 - val_crf_accuracy: 0.9600
Epoch 7/10
 32/928 [>.........

KeyboardInterrupt: 

In [14]:
def training_vis(hist):
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']
    acc = hist.history['crf_accuracy']
    val_acc = hist.history['val_crf_accuracy']

    # make a figure
    fig = plt.figure(figsize=(8,4))

    # subplot loss
    ax1 = fig.add_subplot(121)
    ax1.plot(loss,label='train_loss')
    ax1.plot(val_loss,label='val_loss')
    ax1.scatter(history.epoch, history.history['loss'], marker='*')
    ax1.scatter(history.epoch, history.history['val_loss'], marker='*')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.set_title('Loss on Training and Validation Data')
    
    ax2 = fig.add_subplot(122)
    ax2.plot(acc,label='train_acc')
    ax2.plot(val_acc,label='val_acc')
    ax2.scatter(history.epoch, history.history['crf_accuracy'], marker='*')
    ax2.scatter(history.epoch, history.history['val_crf_accuracy'])
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Accuracy  on Training and Validation Data')
    
training_vis(history)

NameError: name 'history' is not defined

In [15]:
# Predict on test
test_pred = model.model.predict(np.array(testX), verbose=1)

115/115 [==============================] - 8s 68ms/step


In [16]:
truecnt = 0
falsecnt = 0
for (i, pred) in enumerate(test_pred):
    for j, p in enumerate(pred):
        if np.argmax(testY[i][j]) != 0:
            if np.argmax(p) == np.argmax(testY[i][j]):
                truecnt += 1
            else:
                falsecnt += 1
print(truecnt, falsecnt, truecnt/(truecnt+falsecnt))

1619 3894 0.2936695084346091


In [17]:
model.save_model("model_save/init.h5")

In [18]:
# plot
# plt.style.use("ggplot")
# plt.figure(figsize=(12,12))
# plt.plot(history["acc"])
# plt.plot(history["val_acc"])
# plt.show()

In [19]:
import csv
def save_train_tsv(texts, tags):
    
    file_path = "./data/train.tsv"

    with open(file_path, 'w') as file:
        tsv_writer = csv.writer(file, delimiter='\t')
        
        for text, tag in zip(texts, tags):
            for i in range(len(text)):
                tsv_writer.writerow([text[i], tag[i]])

save_train_tsv(train_texts, train_tags)

NameError: name 'train_texts' is not defined

In [ ]:
def read_tsv():
    
    file_path = "./data/train.tsv"
    with open(file_path, 'r') as file:
        reader = csv.reader(file, delimiter='\t')
        for row in reader:
            print(row)
            break

read_tsv()

In [ ]:
import csv
from bert import modeling

from bert.run_classifier import DataProcessor

data_dir = './data/'

class MyProcessor(DataProcessor):
    

    def get_train_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        return ["0", "1", "2", "3"]

    def _create_examples(self, lines, set_type):
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            if set_type == "test":
                text_a = tokenization.convert_to_unicode(line[0])
                label = "0"
            else:
                text_a = tokenization.convert_to_unicode(line[1])
                label = tokenization.convert_to_unicode(line[0])
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

class EditTsv:
    
    file_path = "./data/"
    train_rate = 0.8
    test_rate = 0.2

    @staticmethod
    def read_tsv():

        with open(EditTsv.file_path + 'train.tsv', 'r') as file:
            reader = csv.reader(tsvfile, delimiter='\t')
            for row in reader:
                print(row)
                input()

    @staticmethod
    def save_train_tsv(texts, tags):
        
        with open(EditTsv.file_path + 'train.tsv', 'w') as file:
            tsv_writer = csv.writer(file, delimiter='\t')
            for text, tag in zip(texts, tags):
                for i in len(text):
                    tsv_writer.writerow([text[i], tag[i]])

    @staticmethod
    def save_dev_tsv(texts, tags):

        with open(file_path + 'dev.tsv', 'w') as file:
            tsv_writer = csv.writer(file, delimiter='\t')
            for text, tag in zip(texts, tags):
                for i in len(text):
                    tsv_writer.writerow([text[i], tag[i]])
    
    @staticmethod
    def save_test_tsv(texts, tags):    
        
        with open(EditTsv.file_path + 'test.tsv', 'w') as file:
            tsv_writer = csv.writer(file, delimiter='\t')
            for text, tag in zip(texts, tags):
                for i in len(text):
                    tsv_writer.writerow([text[i], tag[i]])
                    
processors = {
      "myproc": MyProcess
}